In [1]:
from numpy import hstack, ones, array, mat, tile, reshape, squeeze, eye, asmatrix
from numpy.linalg import inv
from pandas import read_csv, Series 
from scipy.linalg import kron
from scipy.optimize import fmin_bfgs
import numpy as np
import statsmodels.api as sm

Next a callable function is used to produce iteration-by-iteration output when using the non-linear optimizer.

In [2]:
iteration = 0
lastValue = 0
functionCount = 0

def iter_print(params):
    global iteration, lastValue, functionCount
    iteration += 1
    print('Func value: {0:}, Iteration: {1:}, Function Count: {2:}'.format(lastValue, iteration, functionCount))

The GMM objective, which is minimized, is defined next.

In [3]:
def gmm_objective(params, pRets, fRets, Winv, out=False):
    global lastValue, functionCount
    T,N = pRets.shape
    T,K = fRets.shape
    beta = squeeze(array(params[:(N*K)]))
    lam = squeeze(array(params[(N*K):]))
    beta = reshape(beta,(N,K))
    lam = reshape(lam,(K,1))
    betalam = beta @ lam
    expectedRet = fRets @ beta.T
    e = pRets - expectedRet
    instr = tile(fRets,N)
    moments1  = kron(e,ones((1,K)))
    moments1 = moments1 * instr
    moments2 = pRets - betalam.T
    moments = hstack((moments1,moments2))

    avgMoment = moments.mean(axis=0)
    
    J = T * mat(avgMoment) * mat(Winv) * mat(avgMoment).T
    J = J[0,0]
    lastValue = J
    functionCount += 1
    if not out:
        return J
    else:
        return J, moments

The G matrix, which is the derivative of the GMM moments with respect to the parameters, is defined.

In [4]:
def gmm_G(params, pRets, fRets):
    T,N = pRets.shape
    T,K = fRets.shape
    beta = squeeze(array(params[:(N*K)]))
    lam = squeeze(array(params[(N*K):]))
    beta = reshape(beta,(N,K))
    lam = reshape(lam,(K,1))
    G = np.zeros((N*K+K,N*K+N))
    ffp = (fRets.T @ fRets) / T
    G[:(N*K),:(N*K)]=kron(eye(N),ffp)
    G[:(N*K),(N*K):] = kron(eye(N),-lam)
    G[(N*K):,(N*K):] = -beta.T
    
    return G

Next, the data is imported and a subset of the test portfolios is selected to make the estimation faster.

In [5]:
data = read_csv('FamaFrench.csv')

# Split using both named colums and ix for larger blocks
dates = data['date'].values
factors = data[['VWMe','SMB','HML']].values
riskfree = data['RF'].values
portfolios = data.iloc[:,5:].values

T,N = portfolios.shape
portfolios = portfolios[:,np.arange(0,N,2)]
T,N = portfolios.shape
excessRet = portfolios - np.reshape(riskfree,(T,1))
K = np.size(factors,1)

Starting values for the factor loadings and rick premia are estimated using OLS and simple means.

In [6]:
betas = []
for i in range(N):
    res = sm.OLS(excessRet[:,i],sm.add_constant(factors)).fit()
    betas.append(res.params[1:])

avgReturn = excessRet.mean(axis=0)
avgReturn.shape = N,1
betas = array(betas)
res = sm.OLS(avgReturn, betas).fit()
riskPremia = res.params

The starting values are computed the first step estimates are found using the non-linear optimizer. The initial weighting matrix is just the identify matrix.

In [7]:
riskPremia.shape = 3
startingVals = np.concatenate((betas.flatten(),riskPremia))

Winv = np.eye(N*(K+1))
args = (excessRet, factors, Winv)
iteration = 0
functionCount = 0
step1opt = fmin_bfgs(gmm_objective, startingVals, args=args, callback=iter_print)

Func value: 1915.9754143260711, Iteration: 1, Function Count: 129
Func value: 1817.0224257797436, Iteration: 2, Function Count: 215
Func value: 1814.9526091309287, Iteration: 3, Function Count: 301
Func value: 1814.8635519699772, Iteration: 4, Function Count: 387
Func value: 1814.7318742173184, Iteration: 5, Function Count: 430
Func value: 1814.4942013819782, Iteration: 6, Function Count: 473
Func value: 1814.4838554978394, Iteration: 7, Function Count: 559
Func value: 1814.483326791821, Iteration: 8, Function Count: 645
Func value: 1814.483224980472, Iteration: 9, Function Count: 731
Func value: 1814.4830322457515, Iteration: 10, Function Count: 774
Func value: 1814.4830313497584, Iteration: 11, Function Count: 860
Func value: 1814.482995719322, Iteration: 12, Function Count: 989
Func value: 1814.396421614271, Iteration: 13, Function Count: 1247
Func value: 1814.3288153896822, Iteration: 14, Function Count: 1290
Func value: 1814.224346463929, Iteration: 15, Function Count: 1333
Func v

Here we look at the risk premia estimates from the first step (inefficient) estimates.

In [8]:
premia = step1opt[-3:]
premia = Series(premia,index=['VWMe', 'SMB', 'HML'])
print('Annualized Risk Premia (First step)')
print(12 * premia)

Annualized Risk Premia (First step)
VWMe    5.829996
SMB     4.068223
HML     1.680948
dtype: float64


Next the first step estimates are used to estimate the moment conditions which are in-turn used to estimate the optimal weighting matrix for the moment conditions. This is then used as an input for the 2nd-step estimates.

In [9]:
out = gmm_objective(step1opt, excessRet, factors, Winv, out=True)
S = np.cov(out[1].T)
Winv2 = inv(S)
args = (excessRet, factors, Winv2)

iteration = 0
functionCount = 0
step2opt = fmin_bfgs(gmm_objective, step1opt, args=args, callback=iter_print)

Func value: 70.6917827630953, Iteration: 1, Function Count: 129
Func value: 69.26303994431365, Iteration: 2, Function Count: 172
Func value: 67.07244176925596, Iteration: 3, Function Count: 215
Func value: 64.57443516066405, Iteration: 4, Function Count: 258
Func value: 62.64097399603331, Iteration: 5, Function Count: 301
Func value: 60.38315471686493, Iteration: 6, Function Count: 344
Func value: 59.77131466595251, Iteration: 7, Function Count: 387
Func value: 59.01670155187132, Iteration: 8, Function Count: 430
Func value: 58.118248347250066, Iteration: 9, Function Count: 473
Func value: 57.16139626780914, Iteration: 10, Function Count: 516
Func value: 56.54119805493946, Iteration: 11, Function Count: 559
Func value: 55.762612325912364, Iteration: 12, Function Count: 602
Func value: 54.70774340385646, Iteration: 13, Function Count: 645
Func value: 54.16273774492472, Iteration: 14, Function Count: 731
Func value: 53.68443040806662, Iteration: 15, Function Count: 774
Func value: 53.249

Finally the VCV of the parameter estimates is computed.

In [10]:
out = gmm_objective(step2opt, excessRet, factors, Winv2, out=True)
G = gmm_G(step2opt, excessRet, factors)
S = np.cov(out[1].T)
vcv = inv(G @ inv(S) @ G.T)/T

The annualized risk premia and their associated t-stats.

In [11]:
premia = step2opt[-3:]
premia = Series(premia,index=['VWMe', 'SMB', 'HML'])
premia_vcv = vcv[-3:,-3:]
print('Annualized Risk Premia')
print(12 * premia)

premia_stderr = np.diag(premia_vcv)
premia_stderr = Series(premia_stderr,index=['VWMe', 'SMB', 'HML'])
print('T-stats')
print(premia / premia_stderr)

Annualized Risk Premia
VWMe    10.089707
SMB      3.457166
HML      7.620109
dtype: float64
T-stats
VWMe    28.282290
SMB     22.372708
HML     43.791632
dtype: float64
